# Flare advocates analysis notebook

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
%matplotlib notebook

## Creating a configuration file

In [1]:
confFilePath = "./agilepy_conf.yaml"

In [ ]:
AGAnalysis.getConfiguration(
    confFilePath = confFilePath,
    userName = "flare-advocate-surname",
    sourceName = "source-name",
    tmin = 0,
    tmax = 0,
    timetype = "MJD",
    glon = 0,
    glat = 0,
    outputDir = "$HOME/agilepy_analysis",
    verboselvl = 1
)

## Obtaining the AGAnalysis object

In [ ]:
ag = AGAnalysis(confFilePath)

## Sources hypothesis

In [ ]:
sources = ag.loadSourcesFromCatalog("2AGL", rangeDist = (0, 10))

In [ ]:
for s in sources:
    print(s)

## Adding a source at runtime

In [ ]:
newSourceDict = {
    "glon" : 79.3,
    "glat": 0.554,
    "spectrumType" : "PowerLaw",
    "flux": 1.69737e-07,
    "index": 2.1
}

s = ag.addSource("CYGX3", newSourceDict)

print(s)

## Deleting sources
Selection params = [name, dist, flux, sqrtts]

In [ ]:
deletedSources = ag.deleteSources('dist >= 3')

## Selecting sources

In [ ]:
sources = ag.selectSources("flux > 0")

## Free a source's parameter
Freeable params = [flux, index, index1, index2, cutoffEnergy, pivotEnergy, curvature, index2]

In [ ]:
affectedSources = ag.freeSources("flux > 0", "cutoffEnergy", True)

In this example, only one source is affected: the one with spectrum type "PLExpCutoff" (it is the only spectrum type that has the "cutoffEnergy" parameter)

In [ ]:
for source in affectedSources:
    print(source)

## Analysis

In [ ]:
ag.generateMaps()

In [ ]:
ag.displayCtsSkyMaps(smooth=3, catalogRegions="2AGL", catalogRegionsColor="green")

In [ ]:
ag.displayExpSkyMaps()

In [ ]:
ag.displayGasSkyMaps()

In [ ]:
ag.calcBkg("CYGX3", galcoeff = [0.7], pastTimeWindow = 14)

In [ ]:
ag.mle()

In [ ]:
ag.selectSources("sqrtTS > 0")

## Light curve

In [ ]:
lightCurveData = ag.lightCurve("CYGX3", binsize=86400)

In [ ]:
cat $lightCurveData

In [ ]:
ag.displayLightCurve()

## Cleaning up

In [ ]:
ag.deleteAnalysisDir()